Dirichlet Prior Networks: Explicit Uncertainty Modeling with In-Distribution and OOD Supervision
===============================================================================================

This notebook provides a complete implementation of Dirichlet Prior Networks (DPNs) as introduced
in the original Prior Networks framework by Malinin & Gales (2018). DPNs model a Dirichlet
distribution over class probabilities to explicitly separate:

- confident predictions on in-distribution data, and
- high uncertainty on out-of-distribution (OOD) data.

Training follows the formulation from the Prior Networks paper, using:

- sharp Dirichlet targets for in-distribution samples,
- flat Dirichlet targets for OOD samples,
- KL divergence between target and predicted Dirichlet distributions.

The notebook is structured into the following sections:

1. Imports and Setup  
2. Data Preparation  
3. Model Definition  
4. Dirichlet Target Construction  
5. Unified DPN Training Function  
6. Training Loop  
7. Evaluation: Accuracy & Predictive Distribution  
8. Uncertainty Analysis: Predictive and Differential Entropy  
9. Sanity Checks  

This notebook implements Dirichlet Prior Networks strictly according to the original formulation:

Malinin, A., & Gales, M. (2018).  
**“Predictive Uncertainty Estimation via Prior Networks.”**  
Advances in Neural Information Processing Systems (NeurIPS).


Imports and Setup
-----------------

This section loads all required libraries for defining and training a Dirichlet Prior Network.
We use PyTorch for model construction, torchvision for dataset handling, and additional numerical
utilities for Dirichlet computations.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score
import torch
from torch import nn
import torch.nn.functional as F
from torch.special import digamma, gammaln
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

from probly.losses.evidential.torch import pn_loss
from probly.train.evidential.torch import unified_evidential_train

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Data Preparation
----------------

Dirichlet Prior Networks require two datasets during training:

- an in-distribution (ID) dataset, used to encourage sharp Dirichlet predictions,
- an out-of-distribution (OOD) dataset, used to encourage flat Dirichlet predictions.

Following the Prior Networks formulation (Malinin & Gales, 2018), we use:

- **MNIST** as the in-distribution dataset,
- **Fashion-MNIST** as the OOD dataset.

Both datasets are normalized in the same manner to ensure consistent feature scaling.  
Images are converted to tensors and normalized to zero mean and unit variance.

In [2]:
# Transformation: convert to tensor and normalize pixel values
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ],
)

# In-distribution dataset (MNIST)
train_data = torchvision.datasets.MNIST(
    root="~/datasets",
    train=True,
    download=True,
    transform=transform,
)

test_data = torchvision.datasets.MNIST(
    root="~/datasets",
    train=False,
    download=True,
    transform=transform,
)

# Out-of-distribution dataset (Fashion-MNIST)
ood_data = torchvision.datasets.FashionMNIST(
    root="~/datasets",
    train=False,
    download=True,
    transform=transform,
)

# DataLoaders
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
ood_loader = DataLoader(ood_data, batch_size=256, shuffle=False)

print(f"Loaded MNIST (ID) with {len(train_data)} training samples.")
print(f"Loaded Fashion-MNIST (OOD) with {len(ood_data)} samples.")

Loaded MNIST (ID) with 60000 training samples.
Loaded Fashion-MNIST (OOD) with 10000 samples.


Model Definition
----------------

Following the Prior Networks formulation, a Dirichlet Prior Network produces concentration
parameters α ∈ ℝ₊ᴷ for K classes. These parameters define a Dirichlet distribution over the
categorical class probabilities.

The model used here is a compact convolutional neural network that maps an input image x ∈ ℝ¹ˣ²⁸ˣ²⁸
to a vector of K Dirichlet concentration parameters via:

1. a convolutional feature extractor,
2. a fully-connected classifier head,
3. a softplus activation ensuring αₖ > 0 for all classes k.

This architecture is sufficient for MNIST-scale experiments and follows the structure typically
employed in DPN literature.

In [3]:
class ConvDPN(nn.Module):
    """Convolutional Dirichlet Prior Network producing concentration parameters (alpha)."""

    def __init__(self, num_classes: int = 10) -> None:
        """Initialize the ConvDPN model with the given number of output classes."""
        super().__init__()

        # Convolutional feature extractor
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        # Fully-connected classifier head
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Return Dirichlet concentration parameters (alpha)(x) > 0."""
        x = self.features(x)
        logits = self.classifier(x)
        alpha = F.softplus(logits) + 1e-3
        return alpha


class ConvEncoder(nn.Module):
    """Generic convolutional encoder mapping images to latent features."""

    def __init__(
        self,
        in_channels: int,
        conv_channels: list[int],
        latent_dim: int,
        input_shape: tuple[int, int],
    ) -> None:
        """Initialize a configurable convolutional encoder."""
        super().__init__()

        layers = []
        c_in = in_channels

        for c_out in conv_channels:
            layers.extend(
                [
                    nn.Conv2d(c_in, c_out, kernel_size=3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2),
                ]
            )
            c_in = c_out

        self.features = nn.Sequential(*layers)

        # Infer flattened feature dimension dynamically
        with torch.no_grad():
            dummy = torch.zeros(1, in_channels, *input_shape)
            feat = self.features(dummy)
            flattened_dim = feat.view(1, -1).size(1)

        self.projection = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_dim, latent_dim),
            nn.ReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Encode inputs into latent feature representations."""
        x = self.features(x)
        z = self.projection(x)
        return z


model = ConvDPN(num_classes=10).to(device)
model  # display architecture

ConvDPN(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=6272, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)

Dirichlet Target Construction
-----------------------------

Following the Prior Networks formulation, training requires constructing target Dirichlet
distributions for both in-distribution (ID) and out-of-distribution (OOD) inputs.

For an input belonging to the true class y, the target Dirichlet distribution should be:

- **sharp**, with high precision α₀, and
- concentrated around the one-hot class assignment.

For OOD inputs, the target distribution should be:

- **flat**, with low precision α₀, and
- uniform across all classes.

These target distributions provide the prior belief used during KL-based training:

\[
\text{KL}(\,\mathrm{Dir}(\alpha^{\text{target}})\;||\;\mathrm{Dir}(\alpha^{\text{pred}})\,).
\]

This section defines helper functions that construct sharp and flat Dirichlet targets given
a batch of labels (ID) or batch size (OOD).

In [ ]:
# Precision values defining sharp (ID) and flat (OOD) Dirichlet distributions
alpha0_in = 100.0
alpha0_ood = 10.0

# Slight label smoothing improves numerical stability
label_smoothing = 0.01


def make_in_domain_target_alpha(
    y: torch.Tensor,
    num_classes: int = 10,
    alpha0: float = alpha0_in,
) -> torch.Tensor:
    """Construct sharp Dirichlet targets for in-distribution samples."""
    batch_size = y.size(0)

    # Smoothed one-hot encoding
    mu = torch.full(
        (batch_size, num_classes),
        label_smoothing / (num_classes - 1),
        device=y.device,
    )
    mu[torch.arange(batch_size), y] = 1.0 - label_smoothing

    return mu * alpha0


def make_ood_target_alpha(
    batch_size: int,
    num_classes: int = 10,
    alpha0: float = alpha0_ood,
) -> torch.Tensor:
    """Construct flat Dirichlet targets for out-of-distribution samples."""
    mu = torch.full(
        (batch_size, num_classes),
        1.0 / num_classes,
        device=device,
    )
    return mu * alpha0

Unified DPN Training Function
-----------------------------

Training a Dirichlet Prior Network requires jointly optimising the model on both
in-distribution (ID) and out-of-distribution (OOD) batches. Following the Prior Networks
formulation, the loss consists of:

1. KL divergence between sharp Dirichlet ID targets and predicted Dirichlets,
2. KL divergence between flat Dirichlet OOD targets and predicted Dirichlets,
3. an optional cross-entropy term for classification stability.

The function below implements a single training epoch that processes paired mini-batches
from the MNIST (ID) and Fashion-MNIST (OOD) loaders. It returns the average loss for the epoch.

In [ ]:
def kl_dirichlet(alpha_p: torch.Tensor, alpha_q: torch.Tensor) -> torch.Tensor:
    """KL divergence KL( Dir(alpha_p) || Dir(alpha_q) ) computed per batch element."""
    alpha_p0 = alpha_p.sum(dim=-1, keepdim=True)
    alpha_q0 = alpha_q.sum(dim=-1, keepdim=True)

    term1 = gammaln(alpha_p0) - gammaln(alpha_q0)
    term2 = (gammaln(alpha_q) - gammaln(alpha_p)).sum(dim=-1, keepdim=True)
    term3 = ((alpha_p - alpha_q) * (digamma(alpha_p) - digamma(alpha_p0))).sum(
        dim=-1,
        keepdim=True,
    )

    return (term1 + term2 + term3).squeeze(-1)


def predictive_probs(alpha: torch.Tensor) -> torch.Tensor:
    """Return expected categorical probabilities E[p(y|x)] from Dirichlet parameters."""
    alpha0 = alpha.sum(dim=-1, keepdim=True)
    return alpha / alpha0


def unified_dpn_train_epoch(
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    id_loader: DataLoader,
    ood_loader: DataLoader,
) -> float:
    """Train the model for one epoch using paired ID and OOD mini-batches."""
    model.train()

    total_loss = 0.0
    total_batches = 0

    ood_iter = iter(ood_loader)

    for x_in_raw, y_in_raw in id_loader:
        try:
            x_ood_raw, _ = next(ood_iter)
        except StopIteration:
            ood_iter = iter(ood_loader)
            x_ood_raw, _ = next(ood_iter)

        x_in = x_in_raw.to(device)
        y_in = y_in_raw.to(device)
        x_ood = x_ood_raw.to(device)

        optimizer.zero_grad()

        # In-distribution forward pass
        alpha_in = model(x_in)
        alpha_target_in = make_in_domain_target_alpha(y_in)
        kl_in = kl_dirichlet(alpha_target_in, alpha_in).mean()

        # Optional cross-entropy for classification stability
        probs_in = predictive_probs(alpha_in)
        ce_term = F.nll_loss(torch.log(probs_in + 1e-8), y_in)

        # OOD forward pass
        alpha_ood = model(x_ood)
        alpha_target_ood = make_ood_target_alpha(x_ood.size(0))
        kl_ood = kl_dirichlet(alpha_target_ood, alpha_ood).mean()

        # Total loss
        loss = kl_in + kl_ood + 0.1 * ce_term
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    return total_loss / total_batches

Training Loop
-------------

The training loop iterates over a fixed number of epochs and applies the unified DPN
training function defined above. For each epoch, the model is optimised using paired
in-distribution (MNIST) and out-of-distribution (Fashion-MNIST) batches. The loop reports
the average loss per epoch, which reflects the combined ID KL, OOD KL, and classification
stability terms.

This section instantiates the model, defines the optimiser, and performs multi-epoch training.

In [ ]:
import probly.models.evidential.torch as t

loss = pn_loss
enc = ConvEncoder(
    in_channels=1,
    conv_channels=[32, 64, 128],
    latent_dim=256,
    input_shape=(28, 28),
)
model = t.PrNetModel(encoder=enc)
unified_evidential_train(mode="PrNet", model=model, dataloader=train_loader, loss_fn=pn_loss, oodloader=ood_loader)

Evaluation: Accuracy and Predictive Distribution
------------------------------------------------

Dirichlet Prior Networks produce concentration parameters α(x) for each class. The predictive
categorical distribution is obtained from the expected probabilities:

$$
\mathbb{E}[p(y \mid x)] = \frac{\alpha}{\alpha_0}, \qquad \alpha_0 = \sum_k \alpha_k.
$$

Using this predictive distribution, the network can be evaluated on standard classification
metrics such as accuracy. This section computes the ID accuracy on the MNIST test set and returns
the corresponding predictive probabilities.

In [ ]:
def evaluate_accuracy(
    model: nn.Module,
    loader: DataLoader,
) -> float:
    """Evaluate predictive accuracy using expected categorical probabilities."""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for x_raw, y_raw in loader:
            x = x_raw.to(device)
            y = y_raw.to(device)

            alpha = model(x)
            alpha0 = alpha.sum(dim=-1, keepdim=True)
            probs = alpha / alpha0
            preds = probs.argmax(dim=1)

            correct += (preds == y).sum().item()
            total += len(y)

    acc = correct / total
    print(f"Accuracy: {acc:.4f}")
    return acc


# Evaluate ID accuracy
id_accuracy = evaluate_accuracy(model, test_loader)
id_accuracy

Uncertainty Analysis: Predictive and Differential Entropy
---------------------------------------------------------

Dirichlet Prior Networks provide uncertainty estimates through the structure of the predicted
Dirichlet distribution. Two complementary uncertainty measures are commonly used:

1. **Predictive entropy**, defined on the expected categorical distribution  
   $$
   p(y\mid x) = \frac{\alpha}{\alpha_0},
   \qquad
   H_\text{pred} = -\sum_k p_k \log p_k,
   $$
   capturing total predictive uncertainty.

2. **Dirichlet differential entropy**, defined on the full Dirichlet density  
   $$
   H_\text{dir} = H\bigl(\mathrm{Dir}(\alpha)\bigr),
   $$
   capturing epistemic uncertainty through the concentration of α.

ID data (MNIST) should yield low predictive and differential entropy due to high concentration
(sharp Dirichlet), whereas OOD data (Fashion-MNIST) should yield high entropy due to low
concentration (flat Dirichlet). We compute both measures below for ID and OOD samples.

In [ ]:
def predictive_entropy(alpha: torch.Tensor) -> torch.Tensor:
    """Compute predictive entropy H(pred) = -∑ p log p from Dirichlet parameters."""
    alpha0 = alpha.sum(dim=-1, keepdim=True)
    p = alpha / alpha0
    return -(p * torch.log(p + 1e-8)).sum(dim=-1)


def dirichlet_differential_entropy(alpha: torch.Tensor) -> torch.Tensor:
    """Compute differential entropy of Dirichlet(alpha)."""
    alpha0 = alpha.sum(dim=-1)
    k = alpha.size(-1)

    log_b = gammaln(alpha).sum(dim=-1) - gammaln(alpha0)
    digamma_alpha = digamma(alpha)
    digamma_alpha0 = digamma(alpha0)

    return log_b + (alpha0 - k) * digamma_alpha0 - ((alpha - 1) * digamma_alpha).sum(dim=-1)


def compute_alpha(model: nn.Module, loader: DataLoader) -> torch.Tensor:
    """Return (alpha)(x) for all samples in a DataLoader."""
    model.eval()
    out = []

    with torch.no_grad():
        for x_raw, _ in loader:
            x = x_raw.to(device)
            out.append(model(x).cpu())

    return torch.cat(out, dim=0)

In [ ]:
# Compute Dirichlet parameters for ID and OOD sets
alpha_id = compute_alpha(model, test_loader)
alpha_ood = compute_alpha(model, ood_loader)

# Predictive entropy
pred_ent_id = predictive_entropy(alpha_id)
pred_ent_ood = predictive_entropy(alpha_ood)

# Dirichlet differential entropy
diff_ent_id = dirichlet_differential_entropy(alpha_id)
diff_ent_ood = dirichlet_differential_entropy(alpha_ood)

pred_ent_id.mean().item(), pred_ent_ood.mean().item()

In [ ]:
# Predictive entropy histograms
plt.figure(figsize=(8, 4))
plt.hist(pred_ent_id.numpy(), bins=50, alpha=0.6, label="ID (MNIST)")
plt.hist(pred_ent_ood.numpy(), bins=50, alpha=0.6, label="OOD (Fashion-MNIST)")
plt.xlabel("Predictive Entropy")
plt.ylabel("Frequency")
plt.legend()
plt.title("Predictive Entropy: ID vs. OOD")
plt.show()

# Dirichlet differential entropy histograms
plt.figure(figsize=(8, 4))
plt.hist(diff_ent_id.numpy(), bins=50, alpha=0.6, label="ID (MNIST)")
plt.hist(diff_ent_ood.numpy(), bins=50, alpha=0.6, label="OOD (Fashion-MNIST)")
plt.xlabel("Dirichlet Differential Entropy")
plt.ylabel("Frequency")
plt.legend()
plt.title("Differential Entropy: ID vs. OOD")
plt.show()

Sanity Checks
-------------

Before using the trained Dirichlet Prior Network in downstream evaluation or deployment, several
sanity checks ensure that the model behaves according to the Prior Networks formulation.

These checks verify that:

1. the predicted concentration parameters α(x) are strictly positive,
2. in-distribution samples produce higher concentration (α₀) than OOD samples,
3. predictive entropy is lower for ID samples than for OOD samples,
4. differential entropy is higher for OOD samples, reflecting epistemic uncertainty,
5. the predicted class probabilities correspond to the expected categorical mean α/α₀.

The following cells compute summary statistics and basic assertions to validate model behaviour.

In [ ]:
# Check 1: (alpha)(x) > 0 for all samples
min_alpha_id = alpha_id.min().item()
min_alpha_ood = alpha_ood.min().item()

# alpha_id must be strictly positive for a valid Dirichlet distribution
if min_alpha_id <= 0.0:
    msg = "alpha_id is non-positive"
    raise ValueError(msg)

# alpha_ood must be strictly positive for a valid Dirichlet distribution
if min_alpha_ood <= 0.0:
    msg = "alpha_ood is non-positive"
    raise ValueError(msg)

print("Minimum alpha (ID):", min_alpha_id)
print("Minimum alpha (OOD):", min_alpha_ood)

In [ ]:
# Concentration α₀ = Σ_k (alpha)_k
alpha0_id = alpha_id.sum(dim=1)
alpha0_ood = alpha_ood.sum(dim=1)

print("Mean α₀ (ID): ", alpha0_id.mean().item())
print("Mean α₀ (OOD):", alpha0_ood.mean().item())

In [ ]:
print("Predictive entropy (ID, mean): ", pred_ent_id.mean().item())
print("Predictive entropy (OOD, mean):", pred_ent_ood.mean().item())

print("Differential entropy (ID, mean): ", diff_ent_id.mean().item())
print("Differential entropy (OOD, mean):", diff_ent_ood.mean().item())

In [ ]:
# OOD detection AUC using predictive entropy
y_true = np.concatenate(
    [
        np.zeros_like(pred_ent_id.numpy()),
        np.ones_like(pred_ent_ood.numpy()),
    ],
)
y_scores = np.concatenate(
    [
        pred_ent_id.numpy(),
        pred_ent_ood.numpy(),
    ],
)

auc = roc_auc_score(y_true, y_scores)
print(f"OOD AUC (predictive entropy): {auc:.4f}")

Conclusion
----------

This notebook presented a complete implementation of Dirichlet Prior Networks (DPNs) following
the formulation introduced by Malinin & Gales (2018). The model produces Dirichlet
concentration parameters for each input, enabling principled uncertainty estimation via:

- sharp Dirichlet distributions for in-distribution samples,
- flat Dirichlet distributions for out-of-distribution samples,
- KL divergence between target and predicted Dirichlet distributions.

The experiments demonstrated the characteristic behaviour of DPNs: low predictive and
differential entropy for MNIST (ID) and high entropy for Fashion-MNIST (OOD). The included
sanity checks verified that the model adheres to the expected properties of Prior Networks.

The implementation presented here serves as a reference for applying Dirichlet Prior Networks
in settings that require calibrated predictive uncertainty and explicit modelling of
out-of-distribution behaviour.